<a href="https://colab.research.google.com/github/somiyagawa/GreekCopticMTEval/blob/main/AssessingLLMsInTranslatingCopticAndAncientGreekOstraca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SETUP

In [14]:
#installs
!pip install sacrebleu
!pip install nltk
!pip install pyter3
!pip install rouge
!pip install -U kaleido
!pip install plotly kaleido matplotlib reportlab PyPDF2

#imports
import difflib
import nltk
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pyter
import sacrebleu
import spacy
import string
import subprocess
import time
#
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize
from nltk.translate import meteor_score
from rouge import Rouge
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots
#
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

#metrics
#A. SCHOOL
#B. LEVENSHTEIN
#C. TER
#D.SACREBLEU
#E. METEOR
#F. ROUGE
#***

#A. SCHOOL
def school_metric(base_text, text, weights={'reused_diff': 0.5, 'not_reused_or_present': 1}):
    words1, words2 = set(base_text.split()), set(text.split())
    reused_diff = sum(1 for w1 in words1 if difflib.get_close_matches(w1, words2, n=1, cutoff=0.8) and w1 not in words2)
    not_reused_or_present = len(words1.symmetric_difference(words2))
    score = (weights['reused_diff'] * reused_diff +
             weights['not_reused_or_present'] * not_reused_or_present)
    return score

#B. LEVENSHTEIN
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        s1, s2 = s2, s1
    if not s2:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

#C. TER
def calculate_ter(reference, candidate):
    ref_tokens = reference.split()
    cand_tokens = candidate.split()
    return pyter.ter(cand_tokens, ref_tokens)

#D.SACREBLEU
def calculate_sacrebleu(reference, candidate):
    if isinstance(reference, str):
        reference = [reference]
    bleu_score = sacrebleu.corpus_bleu([candidate], [reference]).score
    return bleu_score

#E. METEOR
def simple_meteor(reference, candidate):
    reference_tokens = nltk.word_tokenize(reference)
    candidate_tokens = nltk.word_tokenize(candidate)
    score = meteor_score.single_meteor_score(reference_tokens, candidate_tokens)
    return score

#F. ROUGE
def calculate_rouge(reference, candidate):
    reference_tokens = ' '.join(nltk.word_tokenize(reference))
    candidate_tokens = ' '.join(nltk.word_tokenize(candidate))
    rouge = Rouge()
    scores = rouge.get_scores(candidate_tokens, reference_tokens)
    rouge_l_score = scores[0]['rouge-l']
    return rouge_l_score['f']

#other functions
def clean_string(input_str):
    return ''.join(char for char in input_str.lower() if char not in string.punctuation)

def compare_texts(base_text, texts):
    def compare_two_texts(text1, text2):
        words1 = text1.split()
        words2 = text2.split()
        reused_diff = []
        not_reused = []
        not_present = []
        for word1 in words1:
            close_matches = difflib.get_close_matches(word1, words2, n=1, cutoff=0.8)
            if close_matches and close_matches[0] != word1:
                reused_diff.append((word1, close_matches[0]))
            if word1 not in words2:
                not_reused.append(word1)
        for word2 in words2:
            if word2 not in words1:
                not_present.append(word2)
        return [reused_diff, not_reused, not_present]
    return [compare_two_texts(base_text, text) for text in texts]

def score_texts(results, weights):
    scored_texts = []
    for result in results:
        reused_diff_count = len(result[0])
        not_reused_count = len(result[1])
        not_present_count = len(result[2])
        score = (weights['reused_diff'] * reused_diff_count +
                 weights['not_reused'] * not_reused_count +
                 weights['not_present'] * not_present_count)
        scored_texts.append(score)
    return scored_texts

def sort_texts_by_score(names, texts, scores):
    sorted_data = sorted(zip(scores, names, texts))
    sorted_scores, sorted_names, sorted_texts = zip(*sorted_data)
    return sorted_scores, sorted_names, sorted_texts

def merge_dfs_on_index(df1, df2):
    merged_df = pd.concat([df1, df2], axis=1)
    return merged_df

def do_tables(tm_input):
    the_tm = tm_input[0]
    ground_truth = tm_input[1]
    translation_list = tm_input[2]
    label_list = tm_input[3]
    function_names = ['school', 'levenshtein', 'ter', 'sacrebleu', 'meteor', 'rouge']
    flip_scaling = [1, 1, 1, 0, 0, 0]
    metric_functions = [school_metric, levenshtein, calculate_ter, calculate_sacrebleu, simple_meteor, calculate_rouge]
    if len(metric_functions) != len(flip_scaling):
        raise ValueError("flip_scaling list must be the same length as metric_functions list.")
    if len(label_list) != len(translation_list):
        raise ValueError("label_list and translation_list must be of the same length.")
    if len(metric_functions) != len(function_names):
        raise ValueError("metric_functions and function_names must be of the same length.")
    df_original = pd.DataFrame(index=label_list, columns=function_names)
    for label, translation in zip(label_list, translation_list):
        for func, func_name in zip(metric_functions, function_names):
            raw_score = func(ground_truth, translation)
            df_original.at[label, func_name] = raw_score
    df_original = df_original.apply(pd.to_numeric)
    df_scaled = pd.DataFrame(index=label_list, columns=function_names)
    for metric, func_name in zip(metric_functions, function_names):
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(df_original[[func_name]])
        if flip_scaling[metric_functions.index(metric)]:
            scaled_data = 1 - scaled_data
        df_scaled[func_name] = scaled_data
    return df_original, df_scaled

def plotly_four_subplots(df1, df2, df3, df4, title="", x_label="Metrics", y_label="Scaled Values"):
    for df in [df1, df2, df3, df4]:
        if df.empty or not all(df.dtypes.apply(lambda x: pd.api.types.is_numeric_dtype(x))):
            raise ValueError("Each DataFrame should not be empty and must contain only numerical data")
    color_map = px.colors.qualitative.Plotly
    fig = make_subplots(rows=2, cols=2, shared_xaxes=True, shared_yaxes=True,
                        subplot_titles=("tm_874362", "tm_874363", "tm_874364", "tm_874365"))
    dfs = [df1, df2, df3, df4]
    positions = [(1, 1), (1, 2), (2, 1), (2, 2)]
    series_to_traces = {}
    color_index = 0
    series_to_color = {}
    for i, df in enumerate(dfs):
        for column in df.columns:
            if column not in series_to_color:
                series_to_color[column] = color_map[color_index % len(color_map)]
                color_index += 1
            color = series_to_color[column]
            trace = go.Scatter(x=df.index, y=df[column], mode='lines+markers', name=column,
                               showlegend=(column not in series_to_traces),
                               legendgroup=column, line=dict(color=color))
            fig.add_trace(trace, row=positions[i][0], col=positions[i][1])
            if column in series_to_traces:
                series_to_traces[column].append(len(fig.data) - 1)
            else:
                series_to_traces[column] = [len(fig.data) - 1]
    fig.update_layout(
        title_text=title,
        xaxis_title=x_label,
        yaxis_title=y_label,
        showlegend=True,
        legend=dict(
            orientation="v",
            x=1.05,
            xanchor="left",
            y=0.5,
            yanchor="middle"
        )
    )
    pdf_file = "4coptic_ostraca.pdf"
    fig.write_image(pdf_file)
    from google.colab import files
    files.download(pdf_file)
    fig.show()

def plotly_four_subplots2(df1, df2, df3, df4, title="", x_label="Metrics", y_label="Scaled Values"):
    for df in [df1, df2, df3, df4]:
        if df.empty or not all(df.dtypes.apply(lambda x: pd.api.types.is_numeric_dtype(x))):
            raise ValueError("Each DataFrame should not be empty and must contain only numerical data")
    color_map = px.colors.qualitative.Plotly
    fig = make_subplots(rows=2, cols=2, shared_xaxes=True, shared_yaxes=True,
                        subplot_titles=("tm_817897", "tm_89219", "tm_89224", "tm_42504"))
    dfs = [df1, df2, df3, df4]
    positions = [(1, 1), (1, 2), (2, 1), (2, 2)]
    series_to_traces = {}
    color_index = 1  # Start from 1 to skip the first color
    series_to_color = {}
    for i, df in enumerate(dfs):
        for column in df.columns:
            if column not in series_to_color:
                series_to_color[column] = color_map[color_index % len(color_map)]
                color_index += 1
            color = series_to_color[column]
            trace = go.Scatter(x=df.index, y=df[column], mode='lines+markers', name=column,
                               showlegend=(column not in series_to_traces),
                               legendgroup=column, line=dict(color=color))
            fig.add_trace(trace, row=positions[i][0], col=positions[i][1])
            if column in series_to_traces:
                series_to_traces[column].append(len(fig.data) - 1)
            else:
                series_to_traces[column] = [len(fig.data) - 1]
    fig.update_layout(
        title_text=title,
        xaxis_title=x_label,
        yaxis_title=y_label,
        showlegend=True,
        legend=dict(
            orientation="v",
            x=1.05,
            xanchor="left",
            y=0.5,
            yanchor="middle"
        )
    )
    pdf_file2 = "4greek_ostraca.pdf"
    fig.write_image(pdf_file2)
    from google.colab import files
    files.download(pdf_file2)
    fig.show()

def transform_dflist_to_results(dflist):
    result_list = []
    combined_models = pd.Index([])
    for df in dflist:
        combined_models = combined_models.union(df.index)
    for metric in dflist[0].columns:
        cumulative_scores = pd.Series(0, index=combined_models)
        model_counts = pd.Series(0, index=combined_models)
        for df in dflist:
            if metric in df.columns:
                valid_models = df.index.intersection(combined_models)

                cumulative_scores = cumulative_scores.astype(float)

                cumulative_scores[valid_models] += df.loc[valid_models, metric]
                model_counts[valid_models] += 1
        mean_scores = cumulative_scores / model_counts
        mean_scores = mean_scores.dropna()
        metric_df = pd.DataFrame({
            'model': mean_scores.index,
            'score': mean_scores.values
        })
        metric_df = metric_df.sort_values(by='score', ascending=False)
        top3_models = ', '.join([f"{row['model']} ({row['score']:.2f})" for _, row in metric_df.head(3).iterrows()])
        all_models_mean_score = metric_df['score'].mean()
        result_list.append({
            'metric': metric,
            'top3models': top3_models,
            'allmodels': f"{all_models_mean_score:.2f}"
        })
    result_df = pd.DataFrame(result_list)
    result_df.columns = ['metric', 'top3models', 'allmodels(meanscore)']
    return result_df

def combine_results_with_labels(table1, label1, table2, label2):
    table1 = table1.set_index('metric')
    table2 = table2.set_index('metric')
    table1.columns = pd.MultiIndex.from_product([[label1], table1.columns])
    table2.columns = pd.MultiIndex.from_product([[label2], table2.columns])
    combined_table = pd.concat([table1, table2], axis=1)
    return combined_table

def calculate_mean_median(df,given_name):
    mean_values = df.mean()
    median_values = df.median()
    result_df = pd.DataFrame([mean_values, median_values], index=[f'{given_name}_mean', f'{given_name}_median'])
    return result_df

#constants
model_names=['coptic_translator','gpt_3.5','gpt_4','gpt_4o','gemini','gemini_advanced','claude_haiku','claude_sonnet','claude_opus']

#4 Coptic texts
#cpt
cpt='⳨ ⲁⲛⲟⲕ ⲡⲉⲧⲣⲟⲥ ⲉⲡⲉⲓⲇⲏ ⲁⲡⲁⲉⲓⲱⲧ ϫⲟⲟⲩⲧ ⲉⲁⲡⲉ ⲛⲣⲟⲩϩⲉ ϩⲁ ⲡϩⲱⲃ ⲙⲡⲕⲩⲙⲏⲗⲓⲟⲛ ⲛⲁⲡⲁ ⲙⲏⲛⲁⲥ ⲉⲓⲥ ⲡⲧⲱϣ ⲁⲩⲧⲁⲁϥ ⲉⲧⲟⲟⲧ ϫⲉ ⲥϩⲁⲓ ⲉⲑⲉ ⲛⲓⲙ ⲉⲩⲁϣⲥ ⲧⲉⲛⲟⲩ ⲕⲟⲩⲱϣ ⲉⲉⲓ ⲉⲛϩⲏⲧ ⲁⲙⲟⲩ ⲕⲟⲩⲱϣ ⲟⲛ ⲛⲧⲁⲉⲓ ϫⲟⲟⲩ ⲛⲧⲁⲉⲓ'
cpt_hum='i am peter my father sent me to ape in the evening regarding the matter of the church vessel of apa menas here is the order that was given to me write whatever manner they want now if you want to come to the north come and if you prefer me to come send for me and i will come'
cpt_hum2=clean_string(cpt_hum)
cpt_tra='i am peter since my father sent me to the head of the evening at the end of the meal also apa minas behold the order was given to me write as to what thing now you wish to come to me come wish you that I come'
cpt_gpt35='and when he had begun to reckon there was brought unto him one which owed him ten thousand talents but forasmuch as he had not to pay his lord commanded him to be sold and his wife and children and all that he had and payment to be made'
cpt_gpt4='i am the doorkeeper of the fold not the sheep who hear the voice of strangers and flee from him but the sheep know his voice'
cpt_gpt4o='i am peter because my father was a sailor in the sea of kymeilion we spent a lot of time in the sea he always said that we should be happy with everything even when our clothes were worn out and we were tired'
cpt_gem='i peter brought these possessions down beneath like the one who dwells in the great necropolis he is not hidden his tomb has been opened may no one disturb him with evil magic may whoever approaches him be consumed by fire may all of them be protected may they be sealed securely alone until eternity to say nothing of all that will be said'
cpt_gemAdv='i am peter the apostle i am writing to you my brothers from babylon on the twentyfifth of the month of epip i want to tell you that we must all love each other and always pray for one another'
cpt_claHai='i peter since my father sent me to care for the matter of the camel and also the matter of abba menas behold the writing has been given to me that i should write whatever is needed now you wish to come to your senses come you also wish what i have said when i came'
cpt_claSon='I, Peter, since my father has ordered me to go to the field to glean the stalks of the emmer wheat, behold, I am going there. Now I want to go in and I want to say the words that I was taught to say.'
cpt_claOpu='I, Petros, since my father sent me to spend the days on the work of the shrine of Apa Menas, see, the border has been given into my hand so that I write in every way you wish. Now, if you want to come northward, come. If you also want that I come, send and I will come.'
groundedc1=[cpt_tra,cpt_claOpu]
tm874362translations=[cpt_tra,cpt_gpt35,cpt_gpt4,cpt_gpt4o,cpt_gem,cpt_gemAdv,cpt_claHai,cpt_claSon,cpt_claOpu]
tm_874362=['874362',cpt_hum2,[clean_string(x) for x in tm874362translations],model_names,len(cpt)]
#cpt2
cpt2='⳨ ⲁⲛⲟⲕ ⲁⲛⲇⲟⲛⲓⲟⲥ ⲉⲧⲥϩⲁⲓ ⲙⲡⲉⲧⲣⲟⲥ ϫⲉ ⲟⲩⲱϣ ⲟⲩⲛ ⲛⲅⲧⲛⲛⲟⲟⲩ ⲛϩⲟⲙⲛⲧ ⲛⲁⲓ ϫⲉ ⲉⲓϣⲱⲛⲉ ⲙⲉⲓ ⲅⲛ ⲟⲩⲛⲟⲩⲙⲉⲥ ⲛⲧⲁϫⲟϥ ⲉⲣⲟⲓ ⲉϣϫⲉ ⲕⲛⲁϯ ⲛⲁⲓ ⲧⲛⲛⲟⲟⲩ ⲡⲱⲓ ⲛⲁⲓ ϫⲟⲉⲓⲥ ϫⲉ ⲙⲉⲓⲅⲛ ⲟⲩ ⲛ ⲧⲁⲃⲟⲕⲥ ⲉⲡϩⲙⲟⲩ ⲉϥⲃⲟⲥⲧ ⲛⲧⲉⲓϩⲉ ⲟⲩϫⲁⲓ ϩⲙ ⲡϫⲟⲉⲓⲥ ⲧⲁⲁⲥ ⲙⲡⲉⲧⲣⲟⲥ ϩⲓⲧⲛ ⲁⲛⲇⲱⲛⲓⲟⲥ ⲡⲉⲧⲛϣⲏⲣⲉ'
cpt2_hum='it is me antonios who is writing to petros please then send me the money for i am sick and I did not find a nummus to spend for me since you are going to give me send me mine my lord for i did not find to add to the salt it is dried out in this way hail in the lord to give to petros from antonios your son'
cpt2_hum2=clean_string(cpt2_hum)
cpt2_tra='i am an adony who wrote to petros if you wish send me a prisoner for i am sick that i may not say to thee any more if you wish to send me to you send me my lord tell me wherefore i shall not serve you in the burning heat thus the salvation in the lord delivered peter over to the hands of his sons'
cpt2_gpt35='when you pray you shall not be as the hypocrites for they love to stand and pray in the synagogues and in the corners of the streets that they may be seen by men most certainly i tell you they have received their reward'
cpt2_gpt4='i am the lord the father and no darkness nor ignorance is within me for i dwell in light and truth this is the place where i will manifest my glory and where the son of the father is eternally'
cpt2_gpt4o='i antonius am writing to petros that there is no news about me but i am very eager to know what you are doing because i love you very much therefore my lord do not abandon me because you are my only hope may god protect you always peace be with you petros your son antonius'
cpt2_gem='I, Anthony, with the inhabitants of the city who are believers, to Jesus with the whoever seeks the holy place to God, to him, to Joseph with the light, we will go together with our brother who is called Peter, his father, to the city which is in'
cpt2_gemAdv='And Anouk and Andonius and also Peter and also, they didn t want to help. And the others said, We should stay here in this place and rest because we re tired and hungry and thirsty. But Peter said to Andonius, Get up and let s go'
cpt2_claHai='i antonius am writing to peter saying please then send me three for i am sick i dont know a certain message that was told to me if you will give them to me send them to me my lord for i dont know what will save me from death that is how health in the lord give it to peter through antonius your son'
cpt2_claSon='i am antonius responding to peter that i also want to go with a few others for i am wealthy and can afford to give you funds if you will allow me my lord to go with you to the feast which is a joyous occasion for the lord antonius the son gave this to peter'
cpt2_claOpu='i am antonios who is writing to petros please send three homnt to me because i am sick i do not have anything to cover myself if you will give to me send them to me know my lord that i do not have anything to put on to go to the salt field while it is cold like this be well in the lord give it to petros through antonios your son'
groundedc2=[cpt2_claOpu]
tm874363translations=[cpt2_tra,cpt2_gpt35,cpt2_gpt4,cpt2_gpt4o,cpt2_gem,cpt2_gemAdv,cpt2_claHai,cpt2_claSon,cpt2_claOpu]
tm_874363=['874363',cpt2_hum2,[clean_string(x) for x in tm874363translations],model_names,len(cpt2)]
#cpt3
cpt3='ⲡⲥⲙⲱⲉⲓ ⲉⲓϣⲓⲛⲉ ⲉⲡⲁⲥⲟⲛ ⲡⲇⲓⲁⲕⲟⲛⲟⲥ ⲙⲡⲣⲕⲛⲁⲣⲓⲕⲉ ⲉⲧⲃⲉ ⲡⲧⲣⲙⲏⲥⲉ ϯⲛⲁⲱⲥⲕ ⲉϫⲟⲟⲩϥ ⲛⲁⲕ ⲁⲛ ϯⲛⲟⲩ ⲣ ⲡⲛⲁ ⲛⲅϯ ⲟⲩⲁⲓⲡⲉ ⲛⲛⲉⲣϣⲓⲛ ⲙⲡⲣⲱⲙⲉ ⲉⲧⲛⲁⲛ ϯ ⲃⲗϫⲉ ⲛⲁⲕ ⲛⲧⲉⲕⲙⲁϫⲉ ⲧⲱⲕ ⲁⲣⲓ ⲧⲁⲅⲁⲡⲏ ⲙⲡⲣⲕⲱ ⲛϣⲟⲩ ϯ ⲧⲁⲓⲡⲉ ⲛⲛⲉⲣϣⲓⲛ ⲛⲁϥ ϫⲉ ⲡⲉⲥⲕⲏⲗⲙⲱⲥ ϩⲟⲥⲉ ⲧⲁⲁⲥ ⲙⲡⲇⲓⲁⲕⲟⲛⲟⲥ ϩⲓⲧⲛ ⲡⲥ ⲙ ⲱⲉ ⲓ '
cpt3_hum='i psmoei i am greeting my brother the deacon do not blame me for the tremissis i will not delay sending it to you now please give the measure of lentils to the man who will bring you this ostracon, according to your own measure be so kind, do not abstain from giving the measure of lentils to him because the travel is difficult give it to the deacon from psmoei'
cpt3_hum2=clean_string(cpt3_hum)
cpt3_tra='my kinsman i will ask you my kinsman do not hide your face from me for i am with you i will not send you away for my kinsman s sake now pour out my complaint to the poor give me a cup of wine to the poor give me your ear to drink and i will give thee my portion'
cpt3_gpt35='but when you pray go into your inner room close your door and pray to your father who is in secret your father who sees in secret will reward you openly'
cpt3_gpt4='the bishop said to the deacon during the preparation of the holy table that indeed we must always remember that god is everpresent in our actions and in our love for each other he must be the foundation of our love and our actions should reflect that love in our community as it is commanded by the deacon'
cpt3_gpt4o='even though the deacon of the church of cana of galilee had scolded you about the manner in which you approached you still did not give up your will remains firm and no one among the people here can match your wisdom and perseverance your love for the lord is evident in all your actions and no one can dispute your devotion for it is truly sincere'
cpt3_gem='they who are silent forever the blessed ones may they protect him with their great power  no evil spirit can approach him for the protection of the three gods surrounds him may no harm come near him may the great protection of isis be upon him like the protection of the three gods forever  beneath the earth anchonios lies hidden'
cpt3_gemAdv='the lord said to satan the accuser where have you come from satan answered the lord from roaming the earth and walking around on it the lord said to satan have you considered my servant job there is no one on earth like him he is blameless and upright a man who fears god and shuns evil satan replied does job fear god for nothing have you not put a hedge around him and his household and everything he has you have blessed the work of his hands so that his flocks and herds are spread throughout the land but now stretch out your hand and strike everything he has and he will surely curse you to your face'
cpt3_claHai='the steward i am searching for my brother the deacon do not be negligent concerning the stranger i will give him shelter with you now i will give drink and bread to the good person who gives rest to you so that you may speak do the good deed do not refuse i entrust the good person to him for his burden is like this the steward gave this to the deacon'
cpt3_claSon='the brother the deacon i inquire about him do not get annoyed about the delay i will not command you now take an example of the good man i urge you do not reject your speech do charity do not overlook this i order the example to him that the skelmos is greater than the gift of the deacon through the brother'
cpt3_claOpu='psmoei greets my brother the deacon do not be negligent about the tremese i will not delay sending it to you now please give a measure of the good man s oil give the wage to you so that you may fill it please do not cause damage give this measure of oil to him because the harvest is difficult give it to the deacon by psmoei'
groundedc3=[cpt3_claOpu]
tm874364translations=[cpt3_tra,cpt3_gpt35,cpt3_gpt4,cpt3_gpt4o,cpt3_gem,cpt3_gemAdv,cpt3_claHai,cpt3_claSon,cpt3_claOpu]
tm_874364=['874364',cpt3_hum2,[clean_string(x) for x in tm874364translations],model_names,len(cpt3)]
#cpt4
cpt4='⳨ ⲉⲡⲉⲓⲇⲏ ⲁⲕⲙⲟⲟϣⲉ ϩⲓⲧⲟⲟⲧ ϫⲉ ⲉⲓⲛⲁϫⲓ ⲛⲉⲃ ⲣⲏⲟⲩⲉ ⲁϩⲓⲗⲓⲁⲥ ⲥⲱⲃⲉ ⲙⲙⲟⲓ ⲉⲧⲃⲉ ⲛⲥⲓⲙⲥⲓⲙ ⲙⲉϥⲧⲁⲁⲩ ⲛⲁⲓ ⲁⲩⲱ ⲟⲛ ⲟⲩⲙⲛⲧ ⲛⲃⲛ ⲛⲉ ⲉⲛⲧⲁⲓϭⲛⲧϥ ⲙⲁⲣⲉϥϫⲟⲟⲩ ⲛⲧⲉⲟⲉⲓⲡⲉ ⲛⲃⲛⲛⲉ ⲙⲛ ⲡⲕⲁⲙⲟⲩⲗ ⲛⲧⲁⲧⲛ ⲛⲟⲟⲩ ⲛⲟⲣⲝ ⲉϣⲱⲡⲉ ⲇⲉ ⲉϥⲧⲙϫⲟⲟⲩ ⲛⲃⲛⲛⲉ ⲙⲡⲣϫⲟⲟⲩ ⲕⲁⲙⲟⲩⲗ ⲙⲟⲛⲟⲛ ⲉⲓϯ ⲙⲙⲟⲟⲩ ⲛⲁⲕ ⲣ ⲡⲛⲁ ⲛⲅϫⲓ ⲡⲛⲉϩ ⲙⲁ ⲡⲁ ⲥⲟⲛ ⲇⲁⲩⲉⲓⲇ ⲛⲅϫⲟⲟⲩϥ ⲛⲁⲓ ⲟⲩϫⲁⲓ ϩⲙ ⲡϫⲟⲉⲓⲥ'
cpt4_hum='since you left me saying i shall take the crops elias laughed at me concerning the vegetables he did not give them to me and as for a measure of dates that i found may he send the measure of dates with the camel in order that i may send the arax and if  he does not send the dates do not send a camel only when i give them to you be so kind and take the oil at my brother david s place and send it to me hail in the lord'
cpt4_hum2=clean_string(cpt4_hum)
cpt4_tra='since you went by me that i might receive a reward elijah laughed me because of symsim and he gave me ten pounds and if i found him let him tell me the number of fifty shekels of silver and the pitcher of gold but if he should not put them on do not send them only i will give them to you and you shall take the oil and give it to me my brother david and you shall send them to me for a little season'
cpt4_gpt35='but you when you fast anoint your head and wash your face so that you are not seen by men to be fasting but by your father who is in secret and your father who sees in secret will reward you openly'
cpt4_gpt4='since you have walked with me that i would receive the inheritance elijah has asked me about the little ones he gave me and also the many good things that i have found let him speak the names of the good ones and the camel that you all saw grazing but if he does not speak the names of the good ones do not say camel only i give them to you take the land my brother david and tell him this health in the lord'
cpt4_gpt4o='since you have given me strength i will tell you all about the prophet elijah who fled because of the fear he had of the people now let us praise god and pray that he may strengthen our hearts and give us courage to face all trials may he give you peace my dear son david in all your endeavors amen'
cpt4_gem='though his possessions were brought down beneath like one who dwells in the great tomb he is not hidden his tomb has been opened may no one disturb him with evil magic may whoever approaches him be consumed by fire may all of them be protected may they be sealed securely alone until eternity to say nothing of all that will be said'
cpt4_gemAdv='my children i am writing to you because your sins have been forgiven on account of his name i am writing to you fathers because you know him who is from the beginning i am writing to you young men because you have overcome the evil one i write to you children because you know thefather i write to you fathers because you know him who is from the beginning i write to you young men because you are strong and the word of god abides in you and you have overcome the evil one do not love the world or the things in the world if anyone loves the world the love of the father is not in him'
cpt4_claHai='since you have walked with me that i would receive the inheritance elijah has asked me about the little ones he gave me and also the many good things that i have found let him speak the names of the good ones and the camel that you all saw grazing but if he does not speak the names of the good ones do not say camel only i give them to you take the land my brother david and tell him this health in the lord'
cpt4_claSon='since you have walked with me saying that i will receive the rivers elijah himself excuses me concerning the movements of the waterwheels and also a multitude of palms which i planted let them say the truth of the palms and the camel which we prepare but if he does not say the truth of the palms do not prepare the camel only i will give them to you in the spirit to receive the eternal life my brother david and rejoice in the lord'
cpt4_claOpu='since you went away from me i will take the garment elias rebuked me about the sesame seeds but he did not give them to me and also there are ten fish that i bought let him send a measure of fish with the container so that we may send them to alexandria but if he does not send benni do not send the container alone i give them to you please take the oil from my brother david and send it to me be well in the lord'
groundedc4=[cpt4_claOpu]
tm874365translations=[cpt4_tra,cpt4_gpt35,cpt4_gpt4,cpt4_gpt4o,cpt4_gem,cpt4_gemAdv,cpt4_claHai,cpt4_claSon,cpt4_claOpu]
tm_874365=['874365',cpt_hum,[clean_string(x) for x in tm874365translations],model_names,len(cpt4)]
#4 Ancient Greek texts
#grk
grk='Παυλεῖνος Σερήνῳ τῷ ἀδελφῷ χαίρειν. καθὼς εγραψές μοι, ἄδελφε, ἐσπούδασα καὶ ἔλαβον καθὼς ἀντὶ τεσσάρων ἀρταβῶν πε τούτων τὴν τειμὴν οὐκ ελ τεσσάρων ἀρταβῶν δραχμῶν δ πολειτ εἴ τι ἄλλο χρείαν ἔχις. ἀσπάζ ερρῶσθαί σε εὔχομαι.'
grk_hum='Paulinus to his brother Serenus, greetings. In conformity with what you have written to me, brother, I have done my best, and I have obtained, in conformity with for four artabas five the price thereof I did not of four artabas 4 drachmas polite . Let me know if in any other way you have need. Greetings to from I pray for your health.'
grk_hum2=clean_string(grk_hum)
grk_gpt35='To Serenus, my brother, greetings. As you wrote to me, brother, I hastened and received the honor for these things, not four artabas but four drachmas. If you need anything else, let me know. I hope you are well. Greetings.'
grk_gpt4='Paulinus to his brother Serenus, greetings. As you wrote to me, brother, I have made the effort and received as much as for four artabas the price, not four artabas of drachmas, I say, if you need anything else. I send my regards, I wish you good health.'
grk_gpt4o_raw='Pauleinos to his brother Serenos, greetings. As you wrote to me, brother, I was diligent and I received accordingly, instead of the four artabas, the price for them in drachmas, not for four artabas as it is sold. If you need anything else. I greet you, and I pray for your good health.'
grk_gpt4o='Paulinus to his brother Serenus, greetings. As you wrote to me, brother, I was diligent and I received accordingly, instead of the four artabas, the price for them in drachmas, not for four artabas as it is sold. If you need anything else. I greet you, and I pray for your good health.'
grk_gem_raw='Paulinus to his brother Serenus, greetings.As you wrote to me, brother, I have taken care of it and received the price of four artabs of [wheat], which is 400 drachmas. I have paid the city tax and the [other] expenses. If you need anything else, let me know. I greet you and wish you good health'
grk_gem='Paulinus to his brother Serenus, greetings.As you wrote to me, brother, I have taken care of it and received the price of four artabs, which is 400 drachmas. I have paid the city tax and the [other] expenses. If you need anything else, let me know. I greet you and wish you good health'
grk_gemAdv_raw='Paulinus to his brother Serenus, greetings.As you wrote to me, brother, I made inquiries and received, as you requested, four artabs of [wheat] for which I paid 400 drachmas. I have sent them to you by [the hands of] the city magistrate. If you have any other need, let me know. I send you my greetings and pray that you are well.'
grk_gemAdv='Paulinus to his brother Serenus, greetings. As you wrote to me, brother, I made inquiries and received, as you requested, four artabs for which I paid 400 drachmas. I have sent them to you by [the hands of] the city magistrate. If you have any other need, let me know. I send you my greetings and pray that you are well.'
grk_claHai_raw='Pavlinus to his brother Serenus, greetings. As you wrote to me, brother, I have been diligent and have received, in place of four artabae, the price of five of these, not four artabae of drachmas. If you have any other need, let me know. I pray for your good health.'
grk_claHai='Paulinus to his brother Serenus, greetings. As you wrote to me, brother, I have been diligent and have received, in place of four artabae, the price of five of these, not four artabae of drachmas. If you have any other need, let me know. I pray for your good health.'
grk_claSon='Paulinus to his brother Serenus, greetings. As you wrote to me, brother, I made an effort and received the value not of four artabas but of these five artabas in drachmas. If you need anything else, let me know. I bid you farewell and pray you are well.'
grk_claOpu='Paulinus to Serenus, his brother, greetings. Just as you wrote to me, brother, I made haste and received for four artabae ... of these the price not ... of four artabae of drachmae ... polit... if you need anything else. Greet ... I pray for your health.'
grk_names=['gpt_3.5','gpt_4','gpt_4o','gemini','gemini_advanced','claude_haiku','claude_sonnet','claude_opus']
tm817897translations=[grk_gpt35,grk_gpt4,grk_gpt4o,grk_gem,grk_gemAdv,grk_claHai,grk_claSon,grk_claOpu]
tm_817897=['817897',grk_hum2,[clean_string(x) for x in tm817897translations],grk_names,len(grk)]
groundedg1=['all']
#grk2
grk2='Ἑρέννιος Σατορνίλῳ τῷ ἀδελφῷ καὶ κυρίωι πλεῖστα χαίρειν· καλῶς ποήσις δοὺς Στύχωι Ναρκίσσου τὸ ἀσπιδίσκιν αὐτοῦ καὶ αἰτήσας αὐτὸν ἅ μοι ὁμολόγησεν μαλθακτέρια δύο καὶ λιβάνιν. ἔσται οἱ ἄρτοι εἰς τὴν ἐνάτην. περὶ τοῦ σιταρίου μονιηνοῦ, παράδος ὅσου θέλῃς. ἐὰν μὲν περισσὸν ἀλλ...'
grk2_hum='Herennios to Satornilus, his brother and lord, many greetings. Please give to Stichus the slave of Narcissus his little shield and collected from him the two shirts and the incense on which he agreed with me. There will be loaves on the ninth. Concerning the food for NN, deliver it at whatever price you wish. But if a surplus...'
grk2_hum2=clean_string(grk2_hum)
grk2_gpt35_raw='To Herennius Saturninus, my brother and lord, greetings. Since Stichus of Narcissus has done well, giving his small shield to him and requesting what he agreed to me, two softer blankets and frankincense. The bread will be ready by the ninth hour. As for the single grain, provide as much as you wish. If there is an excess...'
grk2_gpt35='To Herennios Saturninus, my brother and lord, greetings. Since Stichus of Narcissus has done well, giving his small shield to him and requesting what he agreed to me, two softer blankets and frankincense. The bread will be ready by the ninth hour. As for the single grain, provide as much as you wish. If there is an excess...'
grk2_gpt4_raw='Herenius to his brother and lord Saturnilus, greetings; You would do well to give Stycho the shield of Narcissus and to ask him for the two soft cloths and frankincense he promised me. The bread will be ready on the ninth. Regarding the barley, give as much as you want. If there is an excess...'
grk2_gpt4o_raw='To Herennius Saturnilus, his brother and lord, many greetings. You will do well to give to Stycho, the slave of Narcissus, his small shield and to ask him for the two cushions and the frankincense that he promised me. The loaves will be ready on the ninth. Concerning the wheat of Mouninos, give as much as you wish. If there is extra, ...'
grk2_gem_raw='Herennius to Saturnilus, his brother and lord, much greeting: You did well in giving Narcissus my little shield and asking him for two emollients and frankincense for me. The loaves will be ready by the ninth. As for the monthly wheat, give as much as you like. If there is a surplus, but...'
grk2_gemAdv_raw='Herennius to his brother and master Saturnilus, with many greetings. You did well to give Stychus, son of Narcissus, his small shield and ask him for the two soft cheeses and the incense he promised me. The loaves will be ready on the ninth. As for the wheat of Monius, hand over as much as you wish. If there is an excess, but…'
grk2_claHai_raw='Herennius, to his brother and master Satornilus, very many greetings. You did well in giving Stychus, son of Narcissus, his little shield and in asking him for the two softening agents and the frankincense that he promised me. The loaves will be his on the ninth. Concerning the Monian wheat, hand over as much as you wish. If there is a surplus, but...'
grk2_claSon_raw='Herennius to his brother and lord Satornilos, many greetings. You will do well to give Stychus, the slave of Narcissus, his little shield and to ask him for the two emollients and frankincense that he promised me. The bread will be ready on the 9th. Concerning the monthly grain ration, hand over as much as you wish. If there is a surplus but...'
grk2_claOpu_raw='Herennius, to Satornilos his brother and lord, very many greetings. Please do well by giving to Stychos, the servant of Narcissus, his small shield and requesting from him the two poultices and frankincense which he promised me. There will be loaves of bread for him on the ninth. Concerning the monthly grain provision, give as much as you wish. If there is surplus, but...'
grk2_gpt4='Herenios to his brother and lord Saturnilus, greetings; You would do well to give Stychus the shield of Narcissus and to ask him for the two soft cloths and frankincense he promised me. The bread will be ready on the ninth. Regarding the barley, give as much as you want. If there is an excess...'
grk2_gpt4o='To Herennius Saturnilus, his brother and lord, many greetings. You will do well to give to Stychus, the slave of Narcissus, his small shield and to ask him for the two cushions and the frankincense that he promised me. The loaves will be ready on the ninth. Concerning the wheat of Mouninos, give as much as you wish. If there is extra, ...'
grk2_gem='Herennios to Saturnilus, his brother and lord, much greeting: You did well in giving Narcissus my little shield and asking him for two emollients and frankincense for me. The loaves will be ready by the ninth. As for the monthly wheat, give as much as you like. If there is a surplus, but...'
grk2_gemAdv='Herennios to his brother and master Saturnilus, with many greetings. You did well to give Stychus, son of Narcissus, his small shield and ask him for the two soft cheeses and the incense he promised me. The loaves will be ready on the ninth. As for the wheat of Monius, hand over as much as you wish. If there is an excess, but…'
grk2_claHai='Herennios, to his brother and master Satornilus, very many greetings. You did well in giving Stychus, son of Narcissus, his little shield and in asking him for the two softening agents and the frankincense that he promised me. The loaves will be his on the ninth. Concerning the Monian wheat, hand over as much as you wish. If there is a surplus, but...'
grk2_claSon='Herennios to his brother and lord Satornilos, many greetings. You will do well to give Stychus, the slave of Narcissus, his little shield and to ask him for the two emollients and frankincense that he promised me. The bread will be ready on the 9th. Concerning the monthly grain ration, hand over as much as you wish. If there is a surplus but...'
grk2_claOpu='Herennios, to Satornilos his brother and lord, very many greetings. Please do well by giving to Stychus, the servant of Narcissus, his small shield and requesting from him the two poultices and frankincense which he promised me. There will be loaves of bread for him on the ninth. Concerning the monthly grain provision, give as much as you wish. If there is surplus, but...'
tm89219translations=[grk2_gpt35,grk2_gpt4,grk2_gpt4o,grk2_gem,grk2_gemAdv,grk2_claHai,grk2_claSon,grk2_claOpu]
tm_89219=['89219',grk2_hum2,[clean_string(x) for x in tm89219translations],grk_names,len(grk2)]
#grk3
grk3='Καπίτων Νετίῳ τῷ ἀδελφῷ̣ χαίρειν· πρὸ μὲν παντὸς ἀνανκαῖον ἡγησάμην διʼ ἐπιστολῆς ἀσπάσασθαί σε. ἐπιδὴ σὺ οὐκ ἔμελες ἐν δίκην ὑπάγειν, ἐρωτῶ σε, ἀδελφέ, εἵνα μου προνοήσῃς αν πιθάριν καὶ λαδικηνὸν εἰς ὕδωρ καὶ πειεταστει. ἐρεῖς ὅτει ὁ ἀδελφός σου ἔρχεται καὶ ἅμα σοῦ κατατι η η εἴνα ὁμοῦ καταλύωμεν. ἤκουσα γὰρ ὅτει καὶ σὺ εἰσπλῖς. ἐχάρην oὖν ὅτει σὺν σοὶ ἔρχεται.'
grk3_hum='Capito to Netios his brother, greetings. First of all I thought it necessary to greet you by letter. Since you were not planning to go off for a lawsuit, I ask you, brother, to take care of a storage jar and a ladikenon jar for water, and. You say that your brother is coming and together with you, so that we may settle together. For I heard also that you are sailing in. I was delighted, then, that he is coming with you.'
grk3_hum2=clean_string(grk3_hum)
grk3_gpt35_raw='Greetings to Capitolinus from Nettius, my brother. I thought it necessary before all else to greet you by letter. Since you did not think it necessary to go to court, I ask you, brother, to provide me with a jar and a flask for water and oil, and let it be tested. You will say when your brother is coming, so that we may lodge together. For I heard that you are coming too. I was glad, therefore, when you are coming with him.'
grk3_gpt4_raw='Capiton to his brother Netius, greetings. Above all, I thought it necessary to greet you through a letter. Since you did not care to go to court, I ask you, brother, to arrange for me a large jar and an oil container for water and to tighten it. Let me know when your brother is coming and we can settle down together. I heard that you also arrived. I was therefore glad that he is coming with you.'
grk3_gpt4o_raw='Kapiton to Netios, his brother, greetings. Before all else, I thought it necessary to greet you by letter. Since you did not care to bring the case to court, I ask you, brother, to take care of a jar and a small flask for water and oil for me. You will say that your brother is coming and that you should stay together. For I heard that you are also arriving. I was glad that you are coming together.'
grk3_gem_raw='Kapiton to his brother Netios, greetings: First of all, I considered it necessary to greet you by letter. Since you have not taken care to come to trial, I ask you, brother, to procure for me a pitcher and a ladle for water and a drinking cup. You will say that your brother is coming and that as soon as he arrives, we will stay together. For I heard that you are also sailing in. So I was glad that he is coming with you.'
grk3_gemAdv_raw='Capitón to his brother Netios, greetings. First of all, I felt it necessary to greet you through a letter. Since you were not going to court, I ask you, brother, to take care of me with a little wine and olive oil for water and a little fish sauce. You will say when your brother is coming and at the same time I will pay you so that we can stay together. For I heard that you are sailing too. So I was glad that he is coming with you.'
grk3_claHai_raw='Capiton to his brother Netius, greetings. Before all else, I considered it necessary to greet you by letter. Since you did not care to come to court, I ask you, brother, that you provide for me a little jar and a Laodicean one for water, and a cup. You will say that your brother is coming and at the same time with you, so that we may lodge together. For I have heard that you too are coming in. I rejoiced, therefore, that he is coming with you.'
grk3_claSon_raw='Capiton to his brother Netius, greetings. Before all else, I considered it necessary to greet you by letter. Since you did not intend to go to court, I ask you, brother, to take care for me to have a small jar and a flask of water and bread. You will say that your brother is coming and also staying with you so that we may lodge together. For I have heard that you too are sailing in. I rejoiced then that he is coming with you.'
grk3_claOpu_raw='Capiton to Netius his brother, greetings. Before all else, I considered it necessary to greet you through a letter. Since you were not intending to go to court, I ask you, brother, to provide for me a jar and Laodicean (oil) for water and... You will say that your brother is coming and with you... so that we may lodge together. For I heard that you too are sailing in. Therefore, I rejoiced that he is coming with you.'
grk3_gpt35='Greetings to Capitolinus from Nettios, my brother. I thought it necessary before all else to greet you by letter. Since you did not think it necessary to go to court, I ask you, brother, to provide me with a jar and a flask for water and oil, and let it be tested. You will say when your brother is coming, so that we may lodge together. For I heard that you are coming too. I was glad, therefore, when you are coming with him.'
grk3_gpt4='Capito to his brother Netios, greetings. Above all, I thought it necessary to greet you through a letter. Since you did not care to go to court, I ask you, brother, to arrange for me a large jar and an oil container for water and to tighten it. Let me know when your brother is coming and we can settle down together. I heard that you also arrived. I was therefore glad that he is coming with you.'
grk3_gpt4o='Capiton to Netios, his brother, greetings. Before all else, I thought it necessary to greet you by letter. Since you did not care to bring the case to court, I ask you, brother, to take care of a jar and a small flask for water and oil for me. You will say that your brother is coming and that you should stay together. For I heard that you are also arriving. I was glad that you are coming together.'
grk3_gem='Capiton to his brother Netios, greetings: First of all, I considered it necessary to greet you by letter. Since you have not taken care to come to trial, I ask you, brother, to procure for me a pitcher and a ladle for water and a drinking cup. You will say that your brother is coming and that as soon as he arrives, we will stay together. For I heard that you are also sailing in. So I was glad that he is coming with you.'
grk3_gemAdv='Capiton to his brother Netios, greetings. First of all, I felt it necessary to greet you through a letter. Since you were not going to court, I ask you, brother, to take care of me with a little wine and olive oil for water and a little fish sauce. You will say when your brother is coming and at the same time I will pay you so that we can stay together. For I heard that you are sailing too. So I was glad that he is coming with you.'
grk3_claHai='Capiton to his brother Netios, greetings. Before all else, I considered it necessary to greet you by letter. Since you did not care to come to court, I ask you, brother, that you provide for me a little jar and a Laodicean one for water, and a cup. You will say that your brother is coming and at the same time with you, so that we may lodge together. For I have heard that you too are coming in. I rejoiced, therefore, that he is coming with you.'
grk3_claSon='Capiton to his brother Netios, greetings. Before all else, I considered it necessary to greet you by letter. Since you did not intend to go to court, I ask you, brother, to take care for me to have a small jar and a flask of water and bread. You will say that your brother is coming and also staying with you so that we may lodge together. For I have heard that you too are sailing in. I rejoiced then that he is coming with you.'
grk3_claOpu='Capiton to Netios his brother, greetings. Before all else, I considered it necessary to greet you through a letter. Since you were not intending to go to court, I ask you, brother, to provide for me a jar and Laodicean (oil) for water and... You will say that your brother is coming and with you... so that we may lodge together. For I heard that you too are sailing in. Therefore, I rejoiced that he is coming with you.'
tm89224translations=[grk3_gpt35,grk3_gpt4,grk3_gpt4o,grk3_gem,grk3_gemAdv,grk3_claHai,grk3_claSon,grk3_claOpu]
tm_89224=['89224',grk3_hum2,[clean_string(x) for x in tm89224translations],grk_names,len(grk3)]
#grk4
grk4='Ὡρίων Ἠλίᾳ τῷ ἀδελφῷ χαίρειν. καὶ ἀπόντος μου ἔμαθον τὴν σπουδὴν ἣν ἐποίησας ἡμῖν παρασχὼν ἡμῖν τὰ βουδια εἰς τὴν σποράν. καλῶς οὖν ποιήσεις, ἄδελφε, συνχωρήσας ἡμῖν αὐτὰ ἄλλας ἡμέρας τρεῖς , ἕως ἀπαλλάξωμεν. παρήνγιλα δὲ Εὐδαίμονι, ἵνα λάβῃ παρὰ Διονυσίου χόρτον. ἤνωσα γὰρ αὐτὸν δοῦναί μοι. θαρρῶ οὖν, ἄδελφε, ὅτι οὐκ ἀμελεῖς μου. ἐρρῶσθαί σε εὔχομαι, ἄδελφε. β ἔτους Τῦβι ιη.'
grk4_human='Orion to his brother Helias, greetings. Even in my absence, I learned about the effort you made on our behalf by providing us with the oxen for the sowing. Therefore, you will do well, brother, to allow us to keep them for another three days until we finish. I have instructed Eudaimon to take hay from Dionysios. For I persuaded him to give it to me. So, I am confident, brother, that you will not neglect me. I pray for your health, brother. Year 2, Tybi 18.'
grk4_human2=clean_string(grk4_human)
grk4_gpt35_raw='Orion greets his brother Elijah. And when I was absent, I learned of the effort you made on our behalf by providing us with the cow dung for sowing. Therefore, brother, you will do well by granting us more of it for three other days, until we finish. I have instructed Eudaemon to receive grass from Dionysius for me, for I promised to give it to him. Therefore, brother, I believe you are not neglecting me. I wish you good health, brother. In the second year of Tybi, [year] 19.'
grk4_gpt4_raw='Orion greets his brother Helios. Even in my absence, I have learned of the efforts you made to provide us with the oxen for sowing. You will do well, brother, to allow us three more days with them, until we are finished. I have instructed Eudaimon to receive fodder from Dionysius, for I have persuaded him to give it to me. Therefore, I am confident, brother, that you are not neglecting me. I pray that you are in good health, brother. Year 2, Tybi 18.'
grk4_gpt4o_raw='Orion to Elias, his brother, greetings. Even in my absence, I learned of the effort you made for us by providing us with the oxen for sowing. Therefore, you will do well, brother, to allow us to keep them for another three days until we finish. I have instructed Eudaemon to get fodder from Dionysius. For I have agreed with him to give it to me. I am confident, brother, that you will not neglect me. I pray for your health, brother. Year 2, 18th of Tybi.'
grk4_gem_raw='Orion to his brother Elias, greetings. In my absence, I learned of the diligence you showed us by providing us with the oxen for sowing. You will therefore do well, brother, to grant us an extension of three more days, until we are finished. I have also instructed Eudaimon to receive fodder from Dionysios. For I have arranged for him to give it to me. I am therefore confident, brother, that you will not neglect me. I wish you good health, brother.Year 2, Tubi 18th.'
grk4_gemAdv_raw='Orion to his brother Elias, greetings. Even in my absence, I learned of the diligence you showed us by providing us with the oxen for the sowing. Therefore, brother, you would do well to allow us to keep them for another three days until we are finished. I have instructed Eudaemon to get fodder from Dionysius. For I have agreed with him to give it to me. So I am confident, brother, that you will not neglect me. I pray that you are well, brother. Year 2, Tybi 18.'
grk4_claHai_raw='Orion to his brother Elias, greetings. Even in my absence, I learned of the diligence which you showed in providing us the oxen for the sowing. You will therefore do well, brother, in granting us the use of them for three more days, until we finish. Moreover, I have instructed Eudaimon to take fodder from Dionysius, for I have bidden him to give it to me. I am confident, therefore, brother, that you will not neglect me. I pray for your health, brother. Year 2, Tybi 18.'
grk4_claSon_raw='Horion to his brother Helias, greetings. Even in my absence I learned of the diligence which you showed by providing us the cattle for the sowing. You will do well then, brother, to grant us them for another three days, until we are done. I have instructed Eudaemon to take fodder from Dionysius. For I bade him to give it to me. I am confident then, brother, that you will not neglect me. I pray for your good health, brother. The 18th of Tubi, the 2nd year.'
grk4_claOpu_raw='Horion to Elias his brother, greetings. Even in my absence, I learned of the zeal which you showed to us by providing us with the oxen for the sowing. You will do well, brother, to allow us to have them for three more days until we finish. I have instructed Eudaemon to get fodder from Dionysius. For I persuaded him to give it to me. I am confident therefore, brother, that you will not neglect me. I pray for your health, brother. Year 2, Tybi 18.'
grk4_gpt35='Orion greets his brother Helijah. And when I was absent, I learned of the effort you made on our behalf by providing us with the cow dung for sowing. Therefore, brother, you will do well by granting us more of it for three other days, until we finish. I have instructed Eudaimon to receive grass from Dionysios for me, for I promised to give it to him. Therefore, brother, I believe you are not neglecting me. I wish you good health, brother. In the second year of Tybi, year 19.'
grk4_gpt4='Orion greets his brother Helios. Even in my absence, I have learned of the efforts you made to provide us with the oxen for sowing. You will do well, brother, to allow us three more days with them, until we are finished. I have instructed Eudaimon to receive fodder from Dionysios, for I have persuaded him to give it to me. Therefore, I am confident, brother, that you are not neglecting me. I pray that you are in good health, brother. Year 2, Tybi 18.'
grk4_gpt4o='Orion to Helias, his brother, greetings. Even in my absence, I learned of the effort you made for us by providing us with the oxen for sowing. Therefore, you will do well, brother, to allow us to keep them for another three days until we finish. I have instructed Eudaimon to get fodder from Dionysios. For I have agreed with him to give it to me. I am confident, brother, that you will not neglect me. I pray for your health, brother. Year 2, 18th of Tybi.'
grk4_gem='Orion to his brother Helias, greetings. In my absence, I learned of the diligence you showed us by providing us with the oxen for sowing. You will therefore do well, brother, to grant us an extension of three more days, until we are finished. I have also instructed Eudaimon to receive fodder from Dionysios. For I have arranged for him to give it to me. I am therefore confident, brother, that you will not neglect me. I wish you good health, brother.Year 2, Tubi 18th.'
grk4_gemAdv='Orion to his brother Helias, greetings. Even in my absence, I learned of the diligence you showed us by providing us with the oxen for the sowing. Therefore, brother, you would do well to allow us to keep them for another three days until we are finished. I have instructed Eudaimon to get fodder from Dionysios. For I have agreed with him to give it to me. So I am confident, brother, that you will not neglect me. I pray that you are well, brother. Year 2, Tybi 18.'
grk4_claHai='Orion to his brother Helias, greetings. Even in my absence, I learned of the diligence which you showed in providing us the oxen for the sowing. You will therefore do well, brother, in granting us the use of them for three more days, until we finish. Moreover, I have instructed Eudaimon to take fodder from Dionysius, for I have bidden him to give it to me. I am confident, therefore, brother, that you will not neglect me. I pray for your health, brother. Year 2, Tybi 18.'
grk4_claSon='Orion to his brother Helias, greetings. Even in my absence I learned of the diligence which you showed by providing us the cattle for the sowing. You will do well then, brother, to grant us them for another three days, until we are done. I have instructed Eudaimon to take fodder from Dionysios. For I bade him to give it to me. I am confident then, brother, that you will not neglect me. I pray for your good health, brother. The 18th of Tubi, the 2nd year.'
grk4_claOpu='Orion to Elias his brother, greetings. Even in my absence, I learned of the zeal which you showed to us by providing us with the oxen for the sowing. You will do well, brother, to allow us to have them for three more days until we finish. I have instructed Eudaimon to get fodder from Dionysios. For I persuaded him to give it to me. I am confident therefore, brother, that you will not neglect me. I pray for your health, brother. Year 2, Tybi 18.'
tm42504translations=[grk4_gpt35,grk4_gpt4,grk4_gpt4o,grk4_gem,grk4_gemAdv,grk4_claHai,grk4_claSon,grk4_claOpu]
tm_42504=['42504',grk4_human2,[clean_string(x) for x in tm42504translations],grk_names,len(grk4)]

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
dfcopt11,dfcopt12=do_tables(tm_874362)
dfcopt21,dfcopt22=do_tables(tm_874363)
dfcopt31,dfcopt32=do_tables(tm_874364)
dfcopt41,dfcopt42=do_tables(tm_874365)
dfgrk11,dfgrk12=do_tables(tm_817897)
dfgrk21,dfgrk22=do_tables(tm_89219)
dfgrk31,dfgrk32=do_tables(tm_89224)
dfgrk41,dfgrk42=do_tables(tm_42504)
#
cpt_metrics=[dfcopt11,dfcopt21,dfcopt31,dfcopt41]
grk_metrics=[dfgrk11,dfgrk21,dfgrk31,dfgrk41]
#
last_grk_table=transform_dflist_to_results(grk_metrics)
last_cpt_table=transform_dflist_to_results(cpt_metrics)
last_table=combine_results_with_labels(last_cpt_table,'4 Coptic Texts', last_grk_table,'4 Ancient Greek Texts')

# TABLE 1: Raw mean scores

In [3]:
#raw mean scores
last_table

4 Coptic Texts  \
                                                    top3models   
metric                                                           
school       gemini (71.75), gemini_advanced (67.00), gpt_4...   
levenshtein  gemini_advanced (312.50), gpt_4 (237.25), gemi...   
ter          gemini_advanced (1.27), gpt_4 (0.99), gemini (...   
sacrebleu    claude_opus (20.02), claude_haiku (11.52), cop...   
meteor       claude_opus (0.46), claude_haiku (0.35), copti...   
rouge        claude_opus (0.44), claude_haiku (0.37), copti...   

                                  \
            allmodels(meanscore)   
metric                             
school                     58.14   
levenshtein               223.78   
ter                         0.92   
sacrebleu                   5.98   
meteor                      0.23   
rouge                       0.25   

                                         4 Ancient Greek Texts  \
                                                    top3models   
metric                                                           
school          gpt_3.5 (45.38), gemini (40.88), gpt_4 (38.50)   
levenshtein  gpt_3.5 (188.25), gpt_4 (157.25), gemini (156.75)   
ter                gpt_3.5 (0.63), gemini (0.55), gpt_4 (0.53)   
sacrebleu    gpt_4o (39.63), claude_opus (37.18), claude_so...   
meteor       claude_opus (0.67), gemini_advanced (0.65), cl...   
rouge        claude_opus (0.65), gpt_4o (0.64), claude_haik...   

                                  
            allmodels(meanscore)  
metric                            
school                     37.03  
levenshtein               148.16  
ter                         0.50  
sacrebleu                  30.89  
meteor                      0.60  
rouge                       0.59

# INTERACTIVE VERSIONS OF FIGURES 2 & 3

In [4]:
dfs = [calculate_mean_median(dfcopt11,'cpt1'), calculate_mean_median(dfcopt21,'cpt2'), calculate_mean_median(dfcopt31,'cpt3'), calculate_mean_median(dfcopt41,'cpt4'),
       calculate_mean_median(dfgrk11,'gr1'), calculate_mean_median(dfgrk21,'gr2'), calculate_mean_median(dfgrk31,'gr3'), calculate_mean_median(dfgrk41,'gr4')]

In [5]:
def plotly_four_subplots(df1, df2, df3, df4, title="", x_label="Metrics", y_label="Scaled Values"):
    for df in [df1, df2, df3, df4]:
        if df.empty or not all(df.dtypes.apply(lambda x: pd.api.types.is_numeric_dtype(x))):
            raise ValueError("Each DataFrame should not be empty and must contain only numerical data")
    color_map = px.colors.qualitative.Plotly
    fig = make_subplots(rows=2, cols=2, shared_xaxes=True, shared_yaxes=True,
                        subplot_titles=("tm_874362", "tm_874363", "tm_874364", "tm_874365"))
    dfs = [df1, df2, df3, df4]
    positions = [(1, 1), (1, 2), (2, 1), (2, 2)]
    series_to_traces = {}
    color_index = 0
    series_to_color = {}
    for i, df in enumerate(dfs):
        for column in df.columns:
            if column not in series_to_color:
                series_to_color[column] = color_map[color_index % len(color_map)]
                color_index += 1
            color = series_to_color[column]
            trace = go.Scatter(x=df.index, y=df[column], mode='lines+markers', name=column,
                               showlegend=(column not in series_to_traces),
                               legendgroup=column, line=dict(color=color))
            fig.add_trace(trace, row=positions[i][0], col=positions[i][1])
            if column in series_to_traces:
                series_to_traces[column].append(len(fig.data) - 1)
            else:
                series_to_traces[column] = [len(fig.data) - 1]
    fig.update_layout(
        title_text=title,
        xaxis_title=x_label,
        yaxis_title=y_label,
        showlegend=True,
        legend=dict(
            orientation="v",
            x=1.05,
            xanchor="left",
            y=0.5,
            yanchor="middle"
        )
    )

    pdf_file = "4coptic_ostraca.pdf"
    fig.write_image(pdf_file)
    from google.colab import files
    files.download(pdf_file)
    fig.show()

def plotly_four_subplots2(df1, df2, df3, df4, title="", x_label="Metrics", y_label="Scaled Values"):
    for df in [df1, df2, df3, df4]:
        if df.empty or not all(df.dtypes.apply(lambda x: pd.api.types.is_numeric_dtype(x))):
            raise ValueError("Each DataFrame should not be empty and must contain only numerical data")
    color_map = px.colors.qualitative.Plotly
    fig = make_subplots(rows=2, cols=2, shared_xaxes=True, shared_yaxes=True,
                        subplot_titles=("tm_817897", "tm_89219", "tm_89224", "tm_42504"))
    dfs = [df1, df2, df3, df4]
    positions = [(1, 1), (1, 2), (2, 1), (2, 2)]
    series_to_traces = {}
    color_index = 1  # Start from 1 to skip the first color
    series_to_color = {}

    for i, df in enumerate(dfs):
        for column in df.columns:
            if column not in series_to_color:
                series_to_color[column] = color_map[color_index % len(color_map)]
                color_index += 1
            color = series_to_color[column]
            trace = go.Scatter(x=df.index, y=df[column], mode='lines+markers', name=column,
                               showlegend=(column not in series_to_traces),
                               legendgroup=column, line=dict(color=color))
            fig.add_trace(trace, row=positions[i][0], col=positions[i][1])
            if column in series_to_traces:
                series_to_traces[column].append(len(fig.data) - 1)
            else:
                series_to_traces[column] = [len(fig.data) - 1]
    fig.update_layout(
        title_text=title,
        xaxis_title=x_label,
        yaxis_title=y_label,
        showlegend=True,
        legend=dict(
            orientation="v",
            x=1.05,
            xanchor="left",
            y=0.5,
            yanchor="middle"
        )
    )

    pdf_file2 = "4greek_ostraca.pdf"
    fig.write_image(pdf_file2)
    from google.colab import files
    files.download(pdf_file2)
    fig.show()

# Scaled metrics evaluating LLMs MT (Coptic texts)

In [9]:
plotly_four_subplots2(dfgrk12.transpose(),dfgrk22.transpose(),dfgrk32.transpose(),dfgrk42.transpose())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Scaled metrics evaluating LLMs MT (Ancient Greek texts)

In [11]:
plotly_four_subplots(dfcopt12.transpose(),dfcopt22.transpose(),dfcopt32.transpose(),dfcopt42.transpose())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>